In [162]:
import pandas as pd
from finta import TA
from pathlib import Path

df_demo = pd.read_csv(Path('raw_data/SPY_data.csv'))
df_demo_demo = '../raw_data/SPY_data.csv'


In [163]:
def create_df(df_ohlcv):
    # create a base dataframe from an imported csv file
    df = pd.read_csv(Path(f'raw_data/{df_ohlcv}'))
    df = df.drop(columns='Unnamed: 0')
    df.set_index('date', inplace=True)
    return df 

In [164]:
def choose_indicator(filepath, indicator_all_caps):
    #Pick an indicator to addd to base dataframe 

    df = create_df(filepath)
    if indicator_all_caps == 'ATR':
        df['ATR'] = TA.ATR(df)
        
    elif indicator_all_caps == 'BBANDS':
        df[['BBANDS Upper','Moving Average', 'BBands Lower']] = TA.BBANDS(df)

    elif indicator_all_caps == 'VBM':
        df['VBM'] = TA.VBM(df)

    elif indicator_all_caps == 'KAMA':
        df['KAMA'] = TA.VBM

    elif indicator_all_caps == 'DYMI':
        df['DYMI'] = TA.DYMI(df)

    elif indicator_all_caps == 'KC':
        df[['KC Upper', 'KC Lower']] = TA.KC(df)

    elif indicator_all_caps == 'APZ':
        df[['APZ Upper', 'APZ Lower']] = TA.APZ(df)
       
    elif indicator_all_caps == 'MI':
        df['MI'] = TA.MI(df)

    else:
        print("Indicator not a volatility indicator")
         

    return df

In [165]:

def custom_pair_indicators(filepath, indicator_all_caps_1, indicator_all_caps_2):
    # pair any two indicators into a single dataframe
    # need to create two instances of choose indicator function
    df_1 = choose_indicator(filepath, indicator_all_caps_1)
    df_2 = choose_indicator(filepath, indicator_all_caps_2)
    
    df = df_1.merge(df_2, how='inner', right_index=True, left_index=True)
    df = df.drop(columns=['open_y', 'high_y','low_y','close_y', 'volume_y','trade_count_y','vwap_y'])
    df = df.rename(columns={'open_x':'open', 'high_x':'high', 'low_x':'low', 'close_x':'close', 'volume_x':'volume', 'trade_count_x':'trade_count', 'vwap_x':'vwap'})
    return df

In [166]:
def drop_any_column(df, *drop_list):
    # drop any column you like
    for drop_me in drop_list:
        df = df.drop(columns=drop_me)
    return df


In [167]:
def mix_pair_indicators(df_1, df_2):
    # merge two different combinatoins of indicators into one frame
    df = df_1.merge(df_2, how='inner', right_index=True, left_index=True)
    df = df.drop(columns=['open_y', 'high_y','low_y','close_y', 'volume_y','trade_count_y','vwap_y'])
    df = df.rename(columns={'open_x':'open', 'high_x':'high', 'low_x':'low', 'close_x':'close', 'volume_x':'volume', 'trade_count_x':'trade_count', 'vwap_x':'vwap'})
    return df


In [169]:
def apply_rolling_window(df,*columns, period):
    # apply rolling window to any column
    for col in columns:
        df[f'{col}'] = df[f'{col}'].rolling(window=period).mean()
    return df


In [224]:
example = custom_pair_indicators(df_demo_demo, "VBM", 'MI')
example.iloc[20:30]

,open,high,low,close,volume,trade_count,vwap,VBM,MI
date,,,,,,,,,
2015-12-30 05:00:00+00:00,207.09,207.21,205.76,205.93,63317679,237053,206.644516,NaN,NaN
2015-12-31 05:00:00+00:00,205.17,205.89,203.87,203.89,114877859,377772,204.693408,NaN,NaN
2016-01-04 05:00:00+00:00,200.53,201.03,198.59,201.01,222353534,655482,200.645153,NaN,NaN
2016-01-05 05:00:00+00:00,201.39,201.90,200.05,201.35,110845729,418705,201.079710,NaN,NaN
2016-01-06 05:00:00+00:00,198.33,200.06,197.60,198.85,152112604,548381,198.958712,NaN,24.342927
2016-01-07 05:00:00+00:00,195.32,197.44,193.59,193.99,213436023,796441,195.345235,-67.158660,24.448476
2016-01-08 05:00:00+00:00,195.21,195.85,191.58,191.95,209567228,754911,193.646639,-66.775739,24.529962
2016-01-11 05:00:00+00:00,193.05,193.41,189.82,192.11,187941153,701513,191.722893,-67.230381,24.477138
2016-01-12 05:00:00+00:00,193.85,194.55,191.14,193.66,172330490,635729,192.886816,-69.124984,24.443229


In [ ]:
def get_column_range(df,period,*columns):
        # get the rolling range of a column
        for col in columns:
                high = df[f'{col}'].rolling(window=period).max()
                low = df[f'{col}'].rolling(window=period).min()
                df[f'{col} {period} period rolling range'] = high -low
        return df


In [207]:
custom_pair_indicators(df_demo_demo, "BBANDS", "KC")

,open,high,low,close,volume,trade_count,vwap,BBANDS Upper,Moving Average,BBands Lower,KC Upper,KC Lower
date,,,,,,,,,,,,
2015-12-01 05:00:00+00:00,209.42,210.8200,209.1100,210.68,97858418,337780,209.921410,NaN,NaN,NaN,NaN,NaN
2015-12-02 05:00:00+00:00,210.60,211.0000,208.2300,208.54,108069059,367013,209.563055,NaN,NaN,NaN,NaN,NaN
2015-12-03 05:00:00+00:00,208.90,209.1500,204.7511,205.58,166224154,546768,206.878936,NaN,NaN,NaN,NaN,NaN
2015-12-04 05:00:00+00:00,206.10,209.9700,205.9300,209.66,192878747,556731,208.178631,NaN,NaN,NaN,NaN,NaN
2015-12-07 05:00:00+00:00,209.20,209.7295,207.2000,208.27,102027111,374705,208.276128,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-24 04:00:00+00:00,445.82,450.5000,444.7600,450.48,61411569,526865,447.609014,454.669463,433.5475,412.425537,453.317195,421.765195
2022-03-25 04:00:00+00:00,451.09,452.9800,448.4300,452.69,77199328,661033,451.047221,457.036838,434.2945,411.552162,453.821938,424.145938
2022-03-28 04:00:00+00:00,452.04,455.9100,450.0600,455.91,68356914,620868,453.294974,459.967142,435.2585,410.549858,454.851944,426.339944


In [ ]:
def ohlc_indicator_mean(df):
    # combine the ohlc and divide it to get the average of all 4
    #  next we are going to combine both features to get an average
    # the df will return two columns ohlc average and feature average
    ohlc = df[['open', 'high','low','close']].mean()
    features = 


In [ ]:
def 